In [2]:
import torch
from BEATs import BEATs, BEATsConfig
from utils import resample_audio
import pandas as pd 

#file path
audio_path = "files/next_5_seconds-f7924b777e474520a516aa584a41d7e6.wav"

#Model Path
model_path = 'checkpoints/BEATs_iter3_plus_AS2M_finetuned_on_AS2M_cpt2.pt'

# load the fine-tuned checkpoints


In [27]:
checkpoint = torch.load(model_path)
cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])
BEATs_model.eval()

# predict the classification probability of each class
audio_input_16khz = resample_audio(audio_path)

probs = BEATs_model.extract_features(audio_input_16khz, padding_mask=None)[0]

for i, (top5_label_prob, top5_label_idx) in enumerate(zip(*probs.topk(k=5))):
    top5_label = [checkpoint['label_dict'][label_idx.item()] for label_idx in top5_label_idx]
    print(f'Top 5 predicted labels of the {i}th audio are {top5_label} with probability of {top5_label_prob}')

Top 5 predicted labels of the 0th audio are ['/m/09x0r', '/m/05zppz', '/m/02qldy', '/m/01g90h', '/t/dd00125'] with probability of tensor([0.6863, 0.1782, 0.0727, 0.0579, 0.0500], grad_fn=<UnbindBackward0>)


In [19]:
def topk_labels_prob(probs, checkpoint):
    classes = pd.read_csv('labels/class_labels_indices.csv', index_col='index')

    results = []
    for (top_label_prob, top_label_idx) in zip(*probs.topk(k=5)):
        top_label = [checkpoint['label_dict'][label_idx.item()] for label_idx in top_label_idx]
        # Get classes from AudioSet class labels indices
        tags = []
        for c in top_label:
            tag = classes[classes['mid'] == c]['display_name'].values[0]
            tags.append(tag)
        results.append([tags[0], top_label_prob.tolist()[0]])
    return results

In [28]:
result = topk_labels_prob(probs, checkpoint)

In [29]:
result

[['Speech', 0.6863177418708801]]

In [22]:
df = pd.read_csv('labels/class_labels_indices.csv', index_col='index')


In [24]:
df.head()

,mid,display_name
index,,
0,/m/09x0r,Speech
1,/m/05zppz,"Male speech, man speaking"
2,/m/02zsn,"Female speech, woman speaking"
3,/m/0ytgt,"Child speech, kid speaking"
4,/m/01h8n0,Conversation


In [25]:
df.shape

(527, 2)